**Import required libraries and scripts**

In [2]:
#Import required libraries and scripts
#from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *


/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/matplotlib/__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/setuptools/_distutils/version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/sklearn_extra/cluster/_commonnn.py:18: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(sklearn.__version__) < LooseVersion("0.23.0"):
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tony/.conda/envs/dockm8/li

In [3]:
software = Path('/home/tony/DockM8/software')
protein_file = Path('/home/tony/DockM8/wocondock_main/protein.pdb')
ref_file = Path('/home/tony/DockM8/wocondock_main/ref.sdf')
docking_library = Path('/home/tony/DockM8/wocondock_main/library.sdf')
docking_programs = ['GNINA', 'SMINA', 'PLANTS', 'QVINAW', 'QVINA2']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS', 'bestpose_QVINAW', 'bestpose_QVINA2']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'SCORCH', 'vinardo', 'plecscore', 'nnscore', 'KORPL', 'ConvexPLR']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'pkasolver'
# ncpus = int(os.cpu_count()-2)
ncpus = 5
pocket = 'reference'
#Create a temporary folder for all further calculations
w_dir = protein_file.parent
print('The working directory has been set to:', w_dir)
(w_dir / 'temp').mkdir(parents=True, exist_ok=True)
if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, protein_file, 8)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

The working directory has been set to: /home/tony/DockM8/wocondock_main

[2023-Sep-22 11:55:59]: Extracting pocket from /home/tony/DockM8/wocondock_main/protein.pdb using /home/tony/DockM8/wocondock_main/ref.sdf as reference ligand

[2023-Sep-22 11:56:16]: Finished extracting pocket from /home/tony/DockM8/wocondock_main/protein.pdb using /home/tony/DockM8/wocondock_main/ref.sdf as reference ligand
{'center': [16.7, -2.69, 17.61], 'size': [16.0, 16.0, 16.0]}


/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/biopandas/pdb/pandas_pdb.py:631: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if pd_version < LooseVersion("0.17.0"):
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/biopandas/pdb/pandas_pdb.py:641: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  elif pd_version < LooseVersion("0.23.0"):


In [ ]:
if (w_dir / 'temp' / 'final_library.sdf').is_file() == False:
    prepare_library(docking_library, id_column, software, protonation, ncpus)

In [ ]:
docking(w_dir, protein_file, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5)

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(str(w_dir / 'temp' / 'allposes.sdf'), idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

In [ ]:
for metric in clustering_metrics:
        if os.path.isfile(w_dir/'temp'/'clustering/'f'{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, protein_file, all_poses, ncpus)

In [4]:
for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, pocket_definition, software, w_dir / 'temp' / 'clustering' / f'{metric}_clustered.sdf', 'GNINA', ncpus)


FileNotFoundError: [Errno 2] No such file or directory: 'rescoring_functions.json'

In [ ]:
calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)

In [ ]:
apply_consensus_methods(w_dir, 'bestpose', 'method6', ['gnina', 'LinF9'])